In [4]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

In [5]:
SELECTED_DATA = os.path.join('data', 'selected_dataset')
ORIGINAL_DATA = os.path.join('data', 'cleaned_dataset')
PROCESSED_DATASET = os.path.join('data', 'processed_dataset')
SAMPLE_IMAGE = 'data/selected_dataset/normal/DSC03990_5.JPG'

In [11]:
model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
model.summary()

In [ ]:
model.layers[-4].get_weights()

## Sample Image

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
Image.open(SAMPLE_IMAGE)

In [ ]:
image = load_img(SAMPLE_IMAGE, target_size=(150, 150))
image

In [ ]:
image = load_img(SAMPLE_IMAGE, target_size=(150, 150))
image = img_to_array(image)
image = preprocess_input(image)

feature_maps = model.predict(np.array([image]))
feature_maps.shape

In [ ]:
feature_maps[0, :, :, 0]

In [ ]:
# plot first 20 filters
n_filters = 20
# specify subplot and turn of axis
fig, ax = plt.subplots(4, 5)

# show the first 20 feature map
for i in range(n_filters):
    # specify subplot and turn of axis
    ax[i//5, i%5].imshow(feature_maps[0, :, :, i], cmap='gray')
    ax[i//5, i%5].axis('off')


In [ ]:
feature_maps[0, :, :, 0]

## Batch Feature Extraction

In [12]:
def extract_and_export(image_path, export_name):
    batch_images = []
    label = []
    file_names = []
    class_map = {'normal': 0, 'infected': 1}
    for image_class in os.listdir(image_path):
        class_path = os.path.join(image_path, image_class)
        for image_file in os.listdir(class_path):
    #         print(os.path.join(class_path, image_file))
            image = load_img(os.path.join(class_path, image_file), target_size=(150, 150))
            image = img_to_array(image)
            image = preprocess_input(image)
            batch_images.append(image)
            label.append(class_map[image_class])
            file_names.append(image_file)
            
    batch_images = np.array(batch_images)
    features = model.predict(batch_images)
    features = features.reshape((-1, 5*5*2048))

    features_df = pd.DataFrame(features,
                               columns=[f'resnet_{x+1}' for x in range(features.shape[-1])])
    features_df['is_infected'] = label
    features_df.insert(0, 'file_name', file_names)
    # features_df.to_csv(os.path.join('data', export_name+'.csv'), index=False)
    features_df.to_parquet(os.path.join('data', export_name+'.parquet'), index=False)
    # features_df.to_excel(os.path.join('data', 'excel', export_name+'.xlsx'), index=False)
    return features_df

In [13]:
orig_df = extract_and_export(SELECTED_DATA, 'orig_img_feature')

63/63 [==============================] - 90s 1s/step
